In [1]:
# dataframe management
import pandas as pd             


# numerical computation
import numpy as np

from sklearn import preprocessing

# visualization library
import seaborn as sns
sns.set(style="white", color_codes=True)
sns.set_context(rc={"font.family":'sans',"font.size":24,"axes.titlesize":24,"axes.labelsize":24})   


# import matplotlib and allow it to plot inline
import matplotlib.pyplot as plt
%matplotlib inline

# seaborn can generate several warnings, we ignore them
import warnings 
warnings.filterwarnings("ignore")

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

#in order to prin all the columns
pd.set_option('display.max_columns', 100)

In [2]:
#reading the datasets
bonifici = pd.read_csv("../datasets/quiubi_bonifici.csv")
segnalaz = pd.read_csv("../datasets/bonifici_segnalaz.csv")
bonifici.set_index('indice',inplace=True)
segnalaz.set_index('indice',inplace=True)

In [3]:
# dropping columns with useless data
bonifici = bonifici.drop(["CAP", "Servizio", "Status", "Paese", "Provincia", "Nazione", "IDTransazione", "CRO", "Causale", "Valuta", "ProfSicurezza", "NumConto", "ABI", "CAB", "Intestatario", "Indirizzo"], axis=1)
bonifici = bonifici.drop(["MsgErrore", "Nominativo", "TipoOperazione"], axis=1)
segnalaz = segnalaz.drop(["CAP", "Servizio", "Status", "Paese", "Provincia", "Nazione", "IDTransazione", "CRO", "Causale", "Valuta", "ProfSicurezza", "NumConto", "ABI", "CAB", "Intestatario", "Indirizzo"], axis=1)
segnalaz = segnalaz.drop(["MsgErrore", "Nominativo", "TipoOperazione"], axis=1)

In [4]:
# c'è un indice duplicato -> eliminalo
# NB: NON E' IL MODO MIGLIORE (GLI INDICI SONO DUPLICATI, MA LE TRANSAZIONI DIVERSE)
bonifici = bonifici[~bonifici.index.duplicated()]

In [5]:
# datasets merge into bonifici
bonifici["isFraud"] = np.zeros(len(bonifici.index))
for index, row in segnalaz.iterrows():
    if index in bonifici.index:
        bonifici.loc[index, "isFraud"] = 1
    else:
        # print(index)
        bonifici.append(row)

bonifici["isFraud"] = pd.to_numeric(bonifici["isFraud"], downcast='integer')

In [9]:
bonifici["isFraud"].value_counts()

0    482320
1       588
Name: isFraud, dtype: int64

# Min Preprocessing

In [6]:
bonifici.Timestamp = pd.to_datetime(bonifici.Timestamp)
bonifici.NumConfermaSMS = bonifici.NumConfermaSMS.eq('Si').astype(int)

In [7]:
bonifici["Importo"].to_numpy().reshape(-1, 1)

array([[1525. ],
       [4653.9],
       [ 600. ],
       ...,
       [ 500. ],
       [ 279.9],
       [ 488. ]])

In [8]:
x = bonifici[["Importo"]].to_numpy()
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
bonifici[["Importo"]] = x_scaled